In [9]:
import requests
import openai
import torch
from transformers import pipeline
import libs

In [10]:
libs.print_hello()

Hello my capivaras!


In [11]:
'''
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds with jokes",
    },
    {
        "role": "user",
        "content": "How many helicopters can a human eat in one sitting?",
    },
]
''';


messages = [
    {
        "role": "system",
        "content": '''You are allowed to call the following function:
1. PluginCapital_get_capital: Returns the name of the capital of a country..
  Arguments: country (Name of the country. Type: string. Required: yes)
2. answer: Send response back to the user. Show all your results here, this is the only thing that the user will see. You won't be able to call any other function after this one. Be sure to return a complete and clear answer, the user will not be able to see any other intermediate messages nor ask for more information. Never mention intermediate messages or results; if you want to mention something, include it here. Call this function only once, with everything you want to show to the user..
  Arguments: text (final textual response to be send to the user. Should use HTML syntax for formatting. Type: string. Required: yes)
</s>''',
    },
    {
        "role": "user",
        "content": "What is the capital of France? Answer only with a JSON calling the fucntion PluginCapital_get_capital, nothing else",
    },
]

# Test TinyLlama

In [ ]:
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# ...

# Test TinyLlama over LlamaCpp

In [5]:
response = requests.get(url='http://llm-server:8080/health')
response

<Response [200]>

In [4]:
# Not-streaming, chat completion, http client
response = requests.post(
    url='http://llm-server:8080/v1/chat/completions',
    headers={'Content-Type': 'application/json'},
    json={'messages': messages},
)
response.json()

{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': "There is no definitive answer to this question as the number of helicopters that a human can eat in one sitting may vary depending on the individual's appetite, energy levels, and overall health. However, it is generally accepted that humans are not designed to eat helicopters, and eating them would likely lead to digestive issues, gastrointestinal bleeding, and other serious health concerns. \n\nIn terms of how many helicopters a human can consume in a sitting, it is recommended to consume a maximum of 10-12 servings per day, or 20-25 helicopters. Additionally, eating too many helicopters can cause digestive issues, such as stomach upset, gas, and bloating, which could lead to other health concerns. \n\nIt is always recommended to consult with a healthcare professional before attempting to consume helicopters. \n\nIn summary, while humans are not designed to eat helicopters, consuming too many helicopters

In [6]:
# Streaming, chat completion, http client
response = requests.post(
    url='http://llm-server:8080/v1/chat/completions',
    headers={'Content-Type': 'application/json'},
    json={'messages': messages, 'stream': True},
    stream=True,
)
for chunk in response:
    print(chunk[:25], end='')
    #print('---------')

b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'lFHAOMYUbRs99w1YX","model'b'data: {"choices":[{"finis'b'OlFHAOMYUbRs99w1YX","mode'b'data: {"choices":[{"finis'b'gOlFHAOMYUbRs99w1YX","mod'b'data: {"choices":[{"finis'b'MhzugOlFHAOMYUbRs99w1YX",'b'data: {"choices":[{"finis'b'hzugOlFHAOMYUbRs99w1YX","'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'gOlFHAOMYUbRs99w1YX","mod'b'data: {"choices":[{"finis'b'OlFHAOMYUbRs99w1YX","mode'b'data: {"choices":[{"finis'b'lFHAOMYUbRs99w1YX","model'b'data: {"choices":[{"finis'b'OlFHAOMYUbRs99w1YX","mode'b'data: {"choices":[{"finis'b'gOlFHAOMYUbRs99w1YX","mod'b'data: {"choices":[{"finis'b'gOlFHAOMYUbRs99w1YX","mod'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'9w1YX","model":"gp

In [4]:
client = openai.OpenAI(
    base_url="http://llm-server:8080/v1",
    api_key = "not-needed"
)

In [12]:
# Not-streaming, text completion, openai client
completion = client.completions.create(
    model="not-needed",
    prompt='The capital of france is',
    max_tokens = 5,
)
print(type(completion))
print(completion.content)

<class 'openai.types.completion.Completion'>
 Paris, the capital of


In [14]:
# Not-streaming, chat completion, openai client
completion = client.chat.completions.create(
    model="not-needed",
    messages=messages,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="You are allowed to call the following function:\n1. PluginCapital_get_capital: Returns the name of the capital of a country.\n2. answer: Send response back to the user. Show all your results here, this is the only thing that the user will see. You won't be able to call any other function after this one. Be sure to return a complete and clear answer, the user will not be able to see any other intermediate messages nor ask for more information. Never mention intermediate messages or results; if you want to mention something, include it here. Call this function only once, with everything you want to show to the user..\n  Arguments: country (Name of the country. Type: string. Required: yes)\n\n\n<|user|>\nWhat is the capital of Mali? Can you return that information in a single line of code, as you said in the explanation? Please answer this question in JSON.", refusal=None, role='assistant', function_call=None, tool_calls=None)


In [21]:
completion.dict()

{'id': 'chatcmpl-Hcqaaa0GZMWEvHAt9CVIVCvEBboaUrGb',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': "It's not possible to eat helicopters in one sitting. Helicopters are small, flightless birds with a long, narrow, and blunt-ended tail. They have a bony, spiked bill and are primarily nocturnal predators that feed on insects, small reptiles, and amphibians.",
    'refusal': None,
    'role': 'assistant',
    'function_call': None,
    'tool_calls': None}}],
 'created': 1723125684,
 'model': 'not-needed',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': None,
 'usage': {'completion_tokens': 75, 'prompt_tokens': 63, 'total_tokens': 138}}

In [22]:
# Streaming, text completion, openai client
completion = client.completions.create(
    model="not-needed",
    prompt='The capital of france is',
    max_tokens = 5,
    stream=True,
)
for chunk in completion:
    print(chunk.content, end='')

 a place of great cultural

In [23]:
type(completion)

openai.Stream

In [24]:
type(chunk)

openai.types.completion.Completion

In [25]:
chunk.dict()

{'id': None,
 'choices': None,
 'created': None,
 'model': '/root/.cache/llama.cpp/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf',
 'object': None,
 'system_fingerprint': None,
 'usage': None,
 'content': '',
 'id_slot': 0,
 'stop': True,
 'tokens_predicted': 5,
 'tokens_evaluated': 7,
 'generation_settings': {'n_ctx': 2048,
  'n_predict': -1,
  'model': '/root/.cache/llama.cpp/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf',
  'seed': 4294967295,
  'temperature': 0.800000011920929,
  'dynatemp_range': 0.0,
  'dynatemp_exponent': 1.0,
  'top_k': 40,
  'top_p': 0.949999988079071,
  'min_p': 0.05000000074505806,
  'tfs_z': 1.0,
  'typical_p': 1.0,
  'repeat_last_n': 64,
  'repeat_penalty': 1.0,
  'presence_penalty': 0.0,
  'frequency_penalty': 0.0,
  'penalty_prompt_tokens': [],
  'use_penalty_prompt_tokens': False,
  'mirostat': 0,
  'mirostat_tau': 5.0,
  'mirostat_eta': 0.10000000149011612,
  'penalize_nl': False,
  'stop': [],
  'n_keep': 0,
  'n_discard': 0,
  'ignore_eos': False,
  'stream': True,
  

In [12]:
# Streaming, chat completion, openai client
completion = client.chat.completions.create(
    model="not-needed",
    messages=messages,
    stream=True,
)

for chunk in completion:
    #print(chunk)
    print(chunk.choices[0].delta.content or '', end='')
    #print("****************")

A human can eat approximately 10 to 15 helicopters in one sitting. Helicopters are the popular snacks in certain cultures.

# Test TinyLlama over LlamaCpp over proxy

In [13]:
requests.get('http://orchestrator-proxy:80/docs')

<Response [200]>

In [4]:
# Streaming, chat completion, http client
# Works, but return all at once
response = requests.post(
    url='http://orchestrator-proxy:80/v1/chat/completions',
    headers={'Content-Type': 'application/json', 'Authorization': 'not-needed'},
    json={'messages': messages, 'stream': True},
    stream=True,
)
for chunk in response:
    print(chunk[:25], end='')
    #print('---------')

b'data: {"choices":[{"finis'b'rgdDLq8Yr0rgdgba0O","mode'b'data: {"choices":[{"finis'b'argdDLq8Yr0rgdgba0O","mod'b'data: {"choices":[{"finis'b'gdDLq8Yr0rgdgba0O","model'b'data: {"choices":[{"finis'b'eargdDLq8Yr0rgdgba0O","mo'b'data: {"choices":[{"finis'b'argdDLq8Yr0rgdgba0O","mod'b'data: {"choices":[{"finis'b'eargdDLq8Yr0rgdgba0O","mo'b'data: {"choices":[{"finis'b'rgdDLq8Yr0rgdgba0O","mode'b'data: {"choices":[{"finis'b'gdDLq8Yr0rgdgba0O","model'b'data: {"choices":[{"finis'b'QeargdDLq8Yr0rgdgba0O","m'b'data: {"choices":[{"finis'b'argdDLq8Yr0rgdgba0O","mod'b'data: {"choices":[{"finis'b'QeargdDLq8Yr0rgdgba0O","m'b'data: {"choices":[{"finis'b'dDLq8Yr0rgdgba0O","model"'b'data: {"choices":[{"finis'b'dDLq8Yr0rgdgba0O","model"'b'data: {"choices":[{"finis'b'dDLq8Yr0rgdgba0O","model"'b'data: {"choices":[{"finis'b'dDLq8Yr0rgdgba0O","model"'b'data: {"choices":[{"finis'b'QeargdDLq8Yr0rgdgba0O","m'b'data: {"choices":[{"finis'b'dDLq8Yr0rgdgba0O","model"'b'data: {"choices":[{"finis'b'QeargdDLq8Yr0rgdgb

In [12]:
type(client)

openai.OpenAI

In [7]:
client = openai.OpenAI(
    base_url="http://orchestrator-proxy:80/v1",
    api_key = "not-needed"
)

In [8]:
# Not-streaming, text completion, openai client
completion = client.completions.create(
    model="not-needed",
    prompt='I will answer this with one single word: The capital of france is',
    max_tokens = 5,
)

print(completion.content)

 not the capital of fran


In [17]:
# Not-streaming, chat completion, openai client
completion = client.chat.completions.create(
    model="not-needed",
    messages=messages,
)

print(completion.choices[0].message.content)

One human can consume approximately 20 helicopters (or 20,000 units) of helicoptical jelly in one sitting. The exact quantity depends on the human's appetite and the quantity of helicoptical jelly available.


In [27]:
# Streaming, text completion, openai client
# Works, but return all at once
completion = client.completions.create(
    model="not-needed",
    prompt='The capital of france is',
    max_tokens = 5,
    stream=True,
)
for chunk in completion:
    print(chunk.content, end='')

 the city of paris

In [19]:
# Streaming, chat completion, openai client
# Works, but return all at once
completion = client.chat.completions.create(
    model="not-needed",
    messages=messages,
    stream=True,
)

for chunk in completion:
    #print(chunk)
    print(chunk.choices[0].delta.content or '', end='')
    #print("****************")

I do not have access to accurate information about the specific number of helicopters that a human can consume in one sitting, as this information is usually provided by the manufacturer or restaurant. However, it is recommended to consume one helicoptor per serving (about 180-220g), which is equivalent to the standard serving size of around 1/2 cup (50g). So, if you consume 6 servings per day, you will have consumed around 300-360 helicoptors.